## DSCC202-402 Data Science at Scale Final Project
### Tracking Tweet sentiment at scale using a pretrained transformer (classifier)
<p>Consider the following illustration of the end to end system that you will be building.  Each student should do their own work.  The project will demonstrate your understanding of Spark Streaming, the medalion data architecture using Delta Lake, Spark Inference at Scale using an MLflow packaged model as well as Exploritory Data Analysis and System Tracking and Monitoring.</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/pipeline.drawio.png">

<p>
You will be pulling an updated copy of the course GitHub repositiory: <a href="https://github.com/lpalum/dscc202-402-spring2025">The Repo</a>.  

Once you have updated your fork of the repository you should see the following template project that is resident in the final_project directory.
</p>

<img src="https://data-science-at-scale.s3.amazonaws.com/images/notebooks.drawio.png">

<p>
You can then pull your project into the Databrick Workspace using the <a href="https://github.com/apps/databricks">Databricks App on Github</a> or by cloning the repo to your laptop and then uploading the final_project directory and its contents to your workspace using file imports.  Your choice.

<p>
Work your way through this notebook which will give you the steps required to submit a complete and compliant project.  The following illustration and associated data dictionary specifies the transformations and data that you are to generate for each step in the medallion pipeline.
</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/dataframes.drawio.png">

#### Bronze Data - raw ingest
- date - string in the source json
- user - string in the source json
- text - tweet string in the source json
- sentiment - the given sentiment of the text as determined by an unknown model that is provided in the source json
- source_file - the path of the source json file the this row of data was read from
- processing_time - a timestamp of when you read this row from the source json

#### Silver Data - Bronze Preprocessing
- timestamp - convert date string in the bronze data to a timestamp
- mention - every @username mentioned in the text string in the bronze data gets a row in this silver data table.
- cleaned_text - the bronze text data with the mentions (@username) removed.
- sentiment - the given sentiment that was associated with the text in the bronze table.

#### Gold Data - Silver Table Inference
- timestamp - the timestamp from the silver data table rows
- mention - the mention from the silver data table rows
- cleaned_text - the cleaned_text from the silver data table rows
- sentiment - the given sentiment from the silver data table rows
- predicted_score - score out of 100 from the Hugging Face Sentiment Transformer
- predicted_sentiment - string representation of the sentiment
- sentiment_id - 0 for negative and 1 for postive associated with the given sentiment
- predicted_sentiment_id - 0 for negative and 1 for positive assocaited with the Hugging Face Sentiment Transformer

#### Application Data - Gold Table Aggregation
- min_timestamp - the oldest timestamp on a given mention (@username)
- max_timestamp - the newest timestamp on a given mention (@username)
- mention - the user (@username) that this row pertains to.
- negative - total negative tweets directed at this mention (@username)
- neutral - total neutral tweets directed at this mention (@username)
- positive - total positive tweets directed at this mention (@username)

When you are designing your approach, one of the main decisions that you will need to make is how you are going to orchestrate the streaming data processing in your pipeline.  There are several valid approaches to triggering your steams and how you will gate the execution of your pipeline.  Think through how you want to proceed and ask questions if you need guidance. The following references may be helpful:
- [Spark Structured Streaming Programming Guide](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html)
- [Databricks Autoloader - Cloudfiles](https://docs.databricks.com/en/ingestion/auto-loader/index.html)
- [In class examples - Spark Structured Streaming Performance](https://dbc-f85bdc5b-07db.cloud.databricks.com/editor/notebooks/2638424645880316?o=1093580174577663)

### Be sure your project runs end to end when *Run all* is executued on this notebook! (7 points)

### This project is worth 25% of your final grade.
- DSCC-202 Students have 55 possible points on this project (see points above and the instructions below)
- DSCC-402 Students have 60 possible points on this project (one extra section to complete)

In [0]:
%run ./includes/includes

In [0]:
"""
Adding a widget to the notebook to control the clearing of a previous run.
or stopping the active streams using routines defined in the utilities notebook
"""
dbutils.widgets.removeAll()

dbutils.widgets.dropdown("clear_previous_run", "No", ["No","Yes"])
if (getArgument("clear_previous_run") == "Yes"):
    clear_previous_run()
    print("Cleared all previous data.")

dbutils.widgets.dropdown("stop_streams", "No", ["No","Yes"])
if (getArgument("stop_streams") == "Yes"):
    stop_all_streams()
    print("Stopped all active streams.")

dbutils.widgets.dropdown("optimize_tables", "No", ["No","Yes"])
if (getArgument("optimize_tables") == "Yes"):
    # Suck up those small files that we have been appending.
    # Optimize the tables
    optimize_table(BRONZE_DELTA)
    optimize_table(SILVER_DELTA)
    optimize_table(GOLD_DELTA)
    print("Optimized all of the Delta Tables")

## 1.0 Import your libraries here (2 points)
- Are your shuffle partitions consistent with your cluster and your workload?
- Do you have the necessary libraries to perform the required operations in the pipeline/application?

In [0]:
# ==== Cell 1.0: Import Libraries & Configure Shuffle Partitions ====
import time

# Core Spark & Delta
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import (
    StructType, StructField, StringType, TimestampType, IntegerType, FloatType, ArrayType
)
from delta import DeltaTable

# MLflow
import mlflow, mlflow.spark

# Hugging Face
from transformers import pipeline

# Utilities
import emoji, pandas as pd, matplotlib.pyplot as plt, re, datetime

# Tune partitions: 2× cores for better parallelism
num_cores = spark.sparkContext.defaultParallelism
desired_partitions = num_cores * 2
spark.conf.set("spark.sql.shuffle.partitions", desired_partitions)

print(f"✔️ defaultParallelism={num_cores}; shuffle.partitions={desired_partitions}")

## 2.0 Define and execute utility functions (3 points)
- Read the source file directory listing
- Count the source files (how many are there?)
- print the contents of one of the files

In [0]:
# ==== Cell: Data‐Mode Selector + Directory Inspection with Sample Caching ====
import time


# 1) Preserve original path
ORIG_TWEET_SOURCE_PATH = TWEET_SOURCE_PATH

# 2) Reset widget if needed
try:
    dbutils.widgets.remove("use_sample_data")
except:
    pass

# 3) Create dropdown
dbutils.widgets.dropdown(
    name="use_sample_data",
    defaultValue="No",
    choices=["No", "Yes"],
    label="Use sample data?"
)
use_sample = dbutils.widgets.get("use_sample_data")

# 4) Paths + config
SAMPLE_TWEET_PATH = USER_DIR + "sample_tweets/"
SAMPLE_SIZE = 10000
current_source = ORIG_TWEET_SOURCE_PATH

if use_sample == "Yes":
    # check existing
    try:
        existing = dbutils.fs.ls(SAMPLE_TWEET_PATH)
    except:
        existing = []
    if len(existing) == SAMPLE_SIZE:
        print(f"✅ Reusing {len(existing)}-file sample at {SAMPLE_TWEET_PATH}")
    else:
        dbutils.fs.rm(SAMPLE_TWEET_PATH, recurse=True)
        dbutils.fs.mkdirs(SAMPLE_TWEET_PATH)
        all_files = dbutils.fs.ls(ORIG_TWEET_SOURCE_PATH)
        n = min(SAMPLE_SIZE, len(all_files))
        for f in all_files[:n]:
            dbutils.fs.cp(f.path, SAMPLE_TWEET_PATH + f.name)
        print(f"✅ Sampled {n} files into {SAMPLE_TWEET_PATH}")
    current_source = SAMPLE_TWEET_PATH

# 5) Inspect
files = dbutils.fs.ls(current_source)
print(f"\n📂 Found {len(files)} files in {current_source}\n")
for f in files:
    print(f" - {f.name}")
if files:
    sample_path = files[0].path
    print(f"\n🔍 Sample file contents:\n{dbutils.fs.head(sample_path,10000)}")
else:
    print("\n⚠️ No files found to inspect.")

# 6) Override global
TWEET_SOURCE_PATH = current_source

# 7) Summary
mode_str = "SAMPLE" if use_sample=="Yes" else "FULL"
print(f"\n▶️ DATA MODE: {mode_str} data (path: {TWEET_SOURCE_PATH})")

## 3.0 Transform the Raw Data to Bronze Data using a stream  (8 points)
- define the schema for the raw data
- setup a read stream using cloudfiles and the source data format
- setup a write stream using delta lake to append to the bronze delta table
- enforce schema
- allow a new schema to be merged into the bronze delta table
- Use the defined BRONZE_CHECKPOINT and BRONZE_DELTA paths defined in the includes
- name your raw to bronze stream as bronze_stream
- transform the raw data to the bronze data using the data definition at the top of the notebook

In [0]:
# ==== Cell 3.0: Transform Raw → Bronze (streaming) =====
import time
time.sleep(30)

from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import input_file_name, current_timestamp

# Schema
raw_schema = StructType([
    StructField("date",      StringType(), True),
    StructField("user",      StringType(), True),
    StructField("text",      StringType(), True),
    StructField("sentiment", StringType(), True)
])

# Ingest
bronze_stream = (
    spark.readStream
         .format("cloudFiles")
         .option("cloudFiles.format","json")
         .option("cloudFiles.schemaLocation", BRONZE_CHECKPOINT + "/schema")
         .schema(raw_schema)
         .load(TWEET_SOURCE_PATH)
         .select(
             "date","user","text","sentiment",
             input_file_name().alias("source_file"),
             current_timestamp().alias("processing_time")
         )
         .writeStream
         .format("delta")
         .outputMode("append")
         .option("checkpointLocation", BRONZE_CHECKPOINT)
         .option("mergeSchema","true")
         .queryName("bronze_stream")
         .start(BRONZE_DELTA)
)
print(f"▶️ 'bronze_stream' started.")

## 4.0 Transform the Bronze Data to Silver Data using a stream (5 points)
- setup a read stream on your bronze delta table
- setup a write stream to append to the silver delta table
- Use the defined SILVER_CHECKPOINT and SILVER_DELTA paths in the includes
- name your bronze to silver stream as silver_stream
- transform the bronze data to the silver data using the data definition at the top of the notebook

In [0]:
# ==== Cell 4.0: Transform Bronze → Silver (streaming) =====
import time
time.sleep(30)

from pyspark.sql.functions import to_timestamp, explode, expr, regexp_replace, split

bronze_df = (
    spark.readStream
         .format("delta")
         .load(BRONZE_DELTA)
)

silver_df = bronze_df.select(
    to_timestamp("date").alias("timestamp"),
    explode(
      expr(
        "filter(transform(split(text,' '), x -> regexp_replace(x,'[^@A-Za-z0-9_]','')),"+
        " x -> x rlike '^@')"
      )
    ).alias("mention"),
    regexp_replace("text","@\\w+","").alias("cleaned_text"),
    "sentiment"
)

silver_stream = (
    silver_df.writeStream
            .format("delta")
            .outputMode("append")
            .option("checkpointLocation", SILVER_CHECKPOINT)
            .option("mergeSchema","true")
            .queryName("silver_stream")
            .start(SILVER_DELTA)
)
print(f"▶️ 'silver_stream' started.")

## 5.0 Transform the Silver Data to Gold Data using a stream (7 points)
- setup a read stream on your silver delta table
- setup a write stream to append to the gold delta table
- Use the defined GOLD_CHECKPOINT and GOLD_DELTA paths defines in the includes
- name your silver to gold stream as gold_stream
- transform the silver data to the gold data using the data definition at the top of the notebook
- Load the pretrained transformer sentiment classifier from the MODEL_NAME at the production level from the MLflow registry
- Use a spark UDF to parallelize the inference across your silver data

In [0]:
# ===== Cell 5.0: Transform Silver → Gold (streaming) with correct label mapping =====
import time
time.sleep(30)

from pyspark.sql.functions import col, when, lit
from pyspark.sql.types import IntegerType
import mlflow

# 1) Stop any existing Gold stream and clear its checkpoint so we start fresh
stop_named_stream(spark, "gold_stream")
dbutils.fs.rm(GOLD_CHECKPOINT, recurse=True)

# 2) Read Silver as a stream from the very first version
silver_stream_df = (
    spark.readStream
         .format("delta")
         .option("startingVersion", 0)
         .load(SILVER_DELTA)
)

# 3) Load the Hugging Face model (registered under MODEL_NAME) as Spark UDFs
model_uri = f"models:/{MODEL_NAME}/Production"
predict_sentiment = mlflow.pyfunc.spark_udf(spark, model_uri, result_type="string")
predict_score     = mlflow.pyfunc.spark_udf(spark, model_uri, result_type="double")

# 4) Build the Gold DataFrame, mapping POS/NEG/NEU correctly
gold_df = (
    silver_stream_df
      .select("timestamp", "mention", "cleaned_text", "sentiment")
      .withColumn("predicted_sentiment", predict_sentiment(col("cleaned_text")))
      .withColumn("predicted_score", (predict_score(col("cleaned_text")) * 100).cast("float"))
      .withColumn(
          "sentiment_id",
          when(col("sentiment") == "positive", lit(1))
         .when(col("sentiment") == "negative", lit(0))
         .otherwise(lit(None))
         .cast(IntegerType())
      )
      .withColumn(
          "predicted_sentiment_id",
          when(col("predicted_sentiment") == "POS", lit(1))
         .when(col("predicted_sentiment") == "NEG", lit(0))
         .when(col("predicted_sentiment") == "NEU", lit(2))
         .otherwise(lit(None))
         .cast(IntegerType())
      )
)

# 5) Write the Gold stream to Delta
gold_stream = (
    gold_df.writeStream
           .format("delta")
           .outputMode("append")
           .option("checkpointLocation", GOLD_CHECKPOINT)
           .option("mergeSchema", "true")
           .queryName("gold_stream")
           .start(GOLD_DELTA)
)

print(f"▶️ 'gold_stream' started, writing to {GOLD_DELTA}")

## 6.0 Monitor your Streams (5 points)
- Setup a loop that runs at least every 10 seconds
- Print a timestamp of the monitoring query along with the list of streams, rows processed on each, and the processing time on each
- Run the loop until all of the data is processed (0 rows read on each active stream)
- Plot a line graph that shows the data processed by each stream over time
- Plot a line graph that shows the average processing time on each stream over time

In [0]:
# ==== Cell 6.0: Robust Stream Monitoring (max 24-hr cap) =====
import time
time.sleep(30)

import pandas as pd
import matplotlib.pyplot as plt

records = []
monitor_start = time.time()
idle_start    = None
max_dur       = 24*3600
idle_thresh   = 60

while True:
    now = pd.Timestamp.now()
    elapsed = time.time()-monitor_start
    active = spark.streams.active
    if not active:
        print("✅ No streams. Exiting.")
        break
    print(f"\n=== Monitoring at {now} (elapsed {elapsed/3600:.2f}h) ===")
    made_prog = False
    for q in active:
        status = q.isActive
        prog   = q.lastProgress
        in_rows = prog.get("numInputRows",0) if prog else 0
        pt_ms   = prog.get("durationMs",{}).get("addBatch") if prog else None
        print(f"• {q.name}: active={status}, rows={in_rows}, time={pt_ms}ms")
        records.append({
            "query":q.name, "monitor_time":now,
            "input_rows":in_rows, "processing_time":pt_ms or 0
        })
        if in_rows>0: made_prog=True

    if made_prog:
        idle_start=None
    else:
        if idle_start is None:
            idle_start=time.time()
        elif time.time()-idle_start>idle_thresh:
            print(f"⚠️ No progress for {idle_thresh}s. Exiting.")
            break

    if elapsed>=max_dur:
        print("⚠️ Hit 24-hr cap. Exiting.")
        break

    time.sleep(10)

stats_df = pd.DataFrame(records)

# Plot 1
plt.figure()
for name,grp in stats_df.groupby("query"):
    plt.plot(grp["monitor_time"],grp["input_rows"],label=name)
plt.xlabel("Time"); plt.ylabel("Rows"); plt.title("Rows Over Time")
plt.legend(); plt.xticks(rotation=45); plt.tight_layout(); plt.show()

# Plot 2
avgp = stats_df.groupby(["monitor_time","query"])["processing_time"].mean().reset_index()
plt.figure()
for name,grp in avgp.groupby("query"):
    plt.plot(grp["monitor_time"],grp["processing_time"],label=name)
plt.xlabel("Time"); plt.ylabel("Proc Time (ms)"); plt.title("Avg Proc Time")
plt.legend(); plt.xticks(rotation=45); plt.tight_layout(); plt.show()

## 7.0 Bronze Data Exploratory Data Analysis (5 points)
- How many tweets are captured in your Bronze Table?
- Are there any columns that contain Nan or Null values?  If so how many and what will you do in your silver transforms to address this?
- Count the number of tweets by each unique user handle and sort the data by descending count.
- How many tweets have at least one mention (@) how many tweet have no mentions (@)
- Plot a bar chart that shows the top 20 tweeters (users)


In [0]:
# ==== Cell 7.0: Bronze EDA =====
import time
time.sleep(30)

from pyspark.sql.functions import col, when, count
import matplotlib.pyplot as plt

bronze_df = spark.read.format("delta").load(BRONZE_DELTA)
total = bronze_df.count()
print(f"Total tweets: {total}")

# Null counts
nulls = bronze_df.select([
    count(when(col(c).isNull(),c)).alias(c) for c in bronze_df.columns
]).collect()[0].asDict()
print("Nulls per column:")
for k,v in nulls.items(): print(f" - {k}: {v}")

# Top users
user_counts = bronze_df.groupBy("user").count().orderBy("count",ascending=False)
print("Top 20 users:")
user_counts.show(20,False)

# Mentions
with_m = bronze_df.filter(col("text").contains("@")).count()
without_m = total-with_m
print(f"With mentions: {with_m}; without: {without_m}")

# Plot top20
top20 = user_counts.limit(20).toPandas()
plt.figure(); plt.bar(top20["user"],top20["count"])
plt.xticks(rotation=90); plt.title("Top 20 Tweeters"); plt.tight_layout(); plt.show()

## 8.0 Capture the accuracy metrics from the gold table in MLflow  (4 points)
Store the following in an MLflow experiment run:
- Store the precision, recall, and F1-score as MLflow metrics
- Store an image of the confusion matrix as an MLflow artifact
- Store the model name and the MLflow version that was used as an MLflow parameters
- Store the version of the Delta Table (input-silver) as an MLflow parameter

In [0]:
# ===== Cell 8.0: MLflow Metrics & Confusion Matrix (multiclass) =====
import time
time.sleep(30)

import mlflow
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
from delta.tables import DeltaTable

# 1) Load Gold and drop nulls
gdf = (spark.read.format("delta")
        .load(GOLD_DELTA)
        .select("sentiment_id", "predicted_sentiment_id")
        .dropna()
)
pdf = gdf.toPandas()
y_true = pdf["sentiment_id"].tolist()
y_pred = pdf["predicted_sentiment_id"].tolist()

# 2) Determine unique classes and compute metrics with a multiclass average
classes = sorted(set(y_true) | set(y_pred))
# Use weighted average to account for class imbalance
precision = precision_score(y_true, y_pred, labels=classes, average="weighted", zero_division=0)
recall    = recall_score(y_true, y_pred, labels=classes, average="weighted", zero_division=0)
f1        = f1_score(y_true, y_pred, labels=classes, average="weighted", zero_division=0)

# 3) Build and save the confusion matrix for all classes
cm = confusion_matrix(y_true, y_pred, labels=classes)

# Map IDs back to labels
label_map = {0: "neg", 1: "pos", 2: "neu"}
tick_labels = [label_map.get(i, str(i)) for i in classes]

fig, ax = plt.subplots()
im = ax.imshow(cm, interpolation="nearest")
ax.set_xticks(range(len(classes)))
ax.set_yticks(range(len(classes)))
ax.set_xticklabels(tick_labels)
ax.set_yticklabels(tick_labels)
for i in range(len(classes)):
    for j in range(len(classes)):
        ax.text(j, i, cm[i, j], ha="center", va="center")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.tight_layout()

cm_path = "/tmp/confusion_matrix.png"
fig.savefig(cm_path)
plt.close(fig)

# 4) Retrieve Silver Delta version
silver_version = DeltaTable.forPath(spark, SILVER_DELTA) \
                    .history(1) \
                    .select("version") \
                    .collect()[0][0]

# 5) Log to MLflow
with mlflow.start_run() as run:
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)
    mlflow.log_artifact(cm_path, artifact_path="confusion_matrix")
    mlflow.log_param("model_name", MODEL_NAME)
    mlflow.log_param("mlflow_version", mlflow.__version__)
    mlflow.log_param("silver_delta_version", silver_version)
    print(f"✅ MLflow run {run.info.run_id}: precision={precision:.3f}, recall={recall:.3f}, f1={f1:.3f}")

## 9.0 Application Data Processing and Visualization (6 points)
- How many mentions are there in the gold data total?
- Count the number of neutral, positive and negative tweets for each mention in new columns
- Capture the total for each mention in a new column
- Sort the mention count totals in descending order
- Plot a bar chart of the top 20 mentions with positive sentiment (the people who are in favor)
- Plot a bar chart of the top 20 mentions with negative sentiment (the people who are the vilians)

*note: A mention is a specific twitter user that has been "mentioned" in a tweet with an @user reference.

In [0]:
# ==== Cell 9.0: Application Aggregation & Visualization =====
import time
time.sleep(30)

from pyspark.sql.functions import col, sum as spark_sum, when
import matplotlib.pyplot as plt

# 1) Load Gold mentions
gd = spark.read.format("delta").load(GOLD_DELTA).select("mention", "predicted_sentiment")
total_mentions = gd.count()
print(f"Total mentions in Gold: {total_mentions}")

# 2) Aggregate counts per mention (adjusting for POS/NEG/NEU labels)
agg = (
    gd.groupBy("mention")
      .agg(
          spark_sum(when(col("predicted_sentiment") == "NEG", 1).otherwise(0)).alias("negative"),
          spark_sum(when(col("predicted_sentiment") == "NEU", 1).otherwise(0)).alias("neutral"),
          spark_sum(when(col("predicted_sentiment") == "POS", 1).otherwise(0)).alias("positive")
      )
      .withColumn("total", col("negative") + col("neutral") + col("positive"))
      .orderBy(col("total").desc())
)

# 3) Show the top 20 mentions by total
print("Top 20 mentions by total tweet count:")
agg.select("mention", "negative", "neutral", "positive", "total") \
   .show(20, truncate=False)

# 4) Bar chart: Top 20 mentions with positive sentiment
top_pos = agg.orderBy(col("positive").desc()).limit(20).toPandas()
plt.figure()
plt.bar(top_pos["mention"], top_pos["positive"])
plt.xlabel("Mention")
plt.ylabel("Count of Positive Tweets")
plt.title("Top 20 Mentions with Positive Sentiment")
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

# 5) Bar chart: Top 20 mentions with negative sentiment
top_neg = agg.orderBy(col("negative").desc()).limit(20).toPandas()
plt.figure()
plt.bar(top_neg["mention"], top_neg["negative"])
plt.xlabel("Mention")
plt.ylabel("Count of Negative Tweets")
plt.title("Top 20 Mentions with Negative Sentiment")
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

## 10.0 Clean up and completion of your pipeline (3 points)
- using the utilities what streams are running? If any.
- Stop all active streams
- print out the elapsed time of your notebook. Note: In the includes there is a variable START_TIME that captures the starting time of the notebook.

In [0]:
# ==== Cell 10.0: Clean Up & Elapsed Time =====
import time
time.sleep(30)

# Active streams
active = spark.streams.active
print("▶️ Active streams:")
for q in active:
    print(f" - {q.name}") if active else print(" None")

# Stop them
stopped = stop_all_streams()
print(f"✅ stop_all_streams() → {stopped}")

# Elapsed
elapsed = time.time() - START_TIME
m,s = divmod(elapsed,60)
print(f"⏱ Total elapsed: {int(m)}m {s:.2f}s")

## 11.0 How Optimized is your Spark Application (Grad Students Only) (5 points)
Graduate students (registered for the DSCC-402 section of the course) are required to do this section.  This is a written analysis using the Spark UI (link to screen shots) that support your analysis of your pipelines execution and what is driving its performance.
Recall that Spark Optimization has 5 significant dimensions of considertation:
- Spill: write to executor disk due to lack of memory
- Skew: imbalance in partition size
- Shuffle: network io moving data between executors (wide transforms)
- Storage: inefficiency due to disk storage format (small files, location)
- Serialization: distribution of code segments across the cluster

Comment on each of the dimentions of performance and how your impelementation is or is not being affected.  Use specific information in the Spark UI to support your description.  

Note: you can take sreenshots of the Spark UI from your project runs in databricks and then link to those pictures by storing them as a publicly accessible file on your cloud drive (google, one drive, etc.)

References:
- [Spark UI Reference Reference](https://spark.apache.org/docs/latest/web-ui.html#web-ui)
- [Spark UI Simulator](https://www.databricks.training/spark-ui-simulator/index.html)

### ENTER YOUR MARKDOWN HERE